# **Introduction to Retrieval Augmented Generation**

We know that LLMs have the capability to generate stuff by themselves. But these tools aren't perfect.

Even though they're super smart, they sometimes get things wrong, especially if they need to be really precise or use the latest information. So, to fix this, some of the brightest minds at Meta AI came up with a new trick called retrieval-augmented generation, or RAG for short, in 2020.

Think of it as giving our language models an assistant. This assistant digs through a massive pile of updated information and feeds the most relevant and recent bits to the LLM.

**Benefits:**  
1. **Enhanced factual accuracy and Domain Specific Expertise:** Imagine a customer service chatbot trained on general conversation data. It might struggle with technical domain specific questions. RAGs allow you to integrate domain-specific knowledge bases, enabling the chatbot to handle these inquires with expertise.
2. **Reduce Hallucination:** LLMs can generate false information, a phenomenon known as hallucination. The knowledge base provided can help support the claims of generative model.

**Components of RAG:**  
1. **Retrieval:** When a user asks a question or provides a prompt, retrievals first help fetch relevant passages from a vast knowledge base. This Knowledge Base could be the company's internal documents, or any other source of text data.
2. **Augmentation:** The retrieved passages are then used to "augment" the LLM's knowledge. This can include various techniques, such as summarization or encoding the key information.
3. **Generation:** Finally LLM leverages its understanding of language along with the augmented information to generate a response. This response can be an answer to a question, a creative text format based on a prompt, etc...

**Applications:**  
1. Question Answering: A RAG powered customer care chatbot can answer customer queries by retrieving product information, FAQs and guides to provide a well-rounded response.
2. Document Summarization: A research paper summarization tool can use RAG to retrieve relevant sections and then generate a summary highlighting main points.
3. Creative Text Generation: A story writing assistant can use RAG to retrieve information about historical periods or fictional creation, helping LLM to generate more deeply engaging stories.
4. Code Generation: A code completion tool can use RAG to retrieve relevant code examples and API documentation, helping developers write code more efficiently.


## **What are Retrievals?**
Understand that the retrievals are specialized in navigating through vast amounts of data to find information that is relevant to a specific query or context. 

Retrieval models focus on the precision of matching query criteria with the data they have access to. Note that retrieval models rely heavily on the quality and structure of the data they access. Their performance depends on the relevance and accuracy of the information stored in the databases they query. 

In simple terms, retrievals search and identify relevant data from a large corpus for a given query.

## **Building a RAG System**
Step 1: Create an Index on available Knowledge Base  
- Data from formats like PDF, HTML, etc is cleaned and converted into plain text. This text is then divided into smaller parts (i.e chunks) and turned into vector representations by passing the chunks into the embedding model to make it easier to find later.

Step 2: Create a Retrival
- When someone asks a question, the RAG system turns that question into vector embedding using the same method used in indexing. Then, it compares this vector to the vectors of the indexed text parts to fing the `k` most similar chunks. These `k` most similar chunks are used in the next step as a context.

Step 3: Generation  
- The system combines the retrieved text parts (i.e. context) with the original question to create a prompt. The LLM uses this prompt to answer the question.

**Step 1: Create an Index on available Knowledge Base**      
1. **Data Collection:** Carefully ingest the data from various sources. This data forms the basis of Knowledge Base.
2. **Split and Parse:** Once the data is ingested, it needs to be broken down into manageable chunks. This is important because the LLMs havve a maximum context wondow that they can process in one go. During this step the data is not only splitted but also parsed to extract the useful metadata. Metadata can be information like document title, authors, etc...
3. **Embedding Generation:** The next step is to convert the chunks into vector embeddings. This can be done using embedding models like BERT, GPT etc... that transforms text into a vector space while capturing semantic relationships and contextual meaning into numberical representation.
4. **Vector Database:** The final step is to store the generated embeddings along with the metadata in a vector database such as ChromaDB, PineCone, etc... These databases are optimized for handing large volumes of data and allow efficient querying.

## **Step 1: Create an Index on available Knowledge Base**

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader('data/subtitles', glob="*.srt", show_progress=True, loader_cls=TextLoader)

docs = loader.load()

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 9100.25it/s]


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

chunks = text_splitter.split_documents(docs)

In [3]:
print("Number of Documents:", len(docs))
print()
print("Number of Chunks:", len(chunks))

Number of Documents: 10

Number of Chunks: 514


In [4]:
from langchain_openai import OpenAIEmbeddings

f = open("keys/.openai_api_key.txt")
OPENAI_API_KEY = f.read()

embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [5]:
# ! pip install langchain_chroma
# Initialize a ChromaDB Connection
from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

In [6]:
db.add_documents(chunks)

['57ca28d3-2051-439c-bdd7-04ee1befbf83',
 'a530d0a7-631a-45dd-8706-11027f04a31e',
 '311688d3-d5d4-4c35-a191-4107e4e45c50',
 '0546bce2-776c-4314-a2e6-52c7fccc69e3',
 'd6bea0b0-3350-497c-90f7-70fc7cd9a319',
 '8cf2f517-744e-4ec1-865b-87969f11014e',
 'a43ecbef-956a-407c-9518-9fc1eb443fae',
 '4d725bd6-89c1-4c62-9ea8-7cd772da8dd5',
 '58017df5-03ff-4f65-b19f-4f418bfe5c31',
 '16439404-a1e8-49d9-84ac-fb010dd2ee9a',
 '7e020630-4d28-4f93-a622-200915ea4d6d',
 '4553e404-20f1-4e61-97fe-9f9a59b09702',
 'ad701168-6427-4c8e-9a2b-700dccff801f',
 '4e93bbb4-0577-4afa-94e8-cc978993218a',
 '09225a97-7071-4fe7-a299-b93cb3b5f095',
 '838c5f48-05eb-4ac9-8a9c-58b3e997ae8a',
 'b246ab5f-1b80-4bde-bd66-c6efb4fbab6f',
 '2b4885f4-058b-4468-a037-cdf715b208cb',
 'd9ad6368-a8f0-4f11-95b6-3b58e451372a',
 'c48eaf23-5741-4354-9081-57b7c9e3017f',
 '1343e426-9714-4658-a4ee-084180ab2dea',
 '53792ea2-d70e-444c-8880-cec77df69ccc',
 '8a460b41-6420-4102-ba49-86b321764998',
 '04e111a2-b5e1-4dd1-b572-91d16ad0b3db',
 '555f8d07-c7e7-

In [7]:
# We can check the already existing values
print(len(db.get()["ids"]))

514


## **Step 2: Create a Retrieval**

In [8]:
query = 'Who is Rachem?'

docs_chroma = db.similarity_search_with_score(query, k=3)

print(len(docs_chroma))

3


In [9]:
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

print(context_text)

250
00:15:17,423 --> 00:15:18,822
What the hell's a Rachem?

251
00:15:19,125 --> 00:15:21,150
Is that a stupid paleontology word...

252
00:15:21,394 --> 00:15:23,726
... I wouldn't know,
because I'm just a waitress?

253
00:15:24,297 --> 00:15:25,787
Rach, come on!

254
00:15:28,001 --> 00:15:29,832
It's "She's not Rachel"!

255
00:15:30,103 --> 00:15:31,934
She's not....

256
00:15:39,913 --> 00:15:41,278
My diary! Brilliant!

242
00:14:53,433 --> 00:14:55,264
"Just a waitress"?

243
00:14:56,569 --> 00:14:58,298
Now that was....

244
00:14:58,738 --> 00:15:00,865
I mean, as opposed to....

245
00:15:02,375 --> 00:15:04,707
Okay, is this over yet? Rach?

246
00:15:05,478 --> 00:15:08,879
I do not have chubby ankles!

247
00:15:09,082 --> 00:15:10,242
No! I

248
00:15:10,483 --> 00:15:13,611
Okay, look at the other side.
Look at Julie's column.

249
00:15:14,487 --> 00:15:15,954
"She's not Rachem"?

46
00:03:06,910 --> 00:03:11,010
I'm the same way.Show me a bottle of wine.
and I'm l

## **Step 3: Generation**

In [10]:
from langchain_core.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

prompt = prompt_template.format(context=context_text, question=query)

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

response_text = model.invoke(prompt)

print(response_text.content)

Rachem is a misspelling or mispronunciation of Rachel, who is a person being referenced in the conversation.
